## Imports

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, Conv2D, MaxPooling1D, BatchNormalization

Using TensorFlow backend.


In [3]:
import tensorflow as tf

In [4]:
from os import system as sys

In [5]:
from sklearn.metrics import confusion_matrix

In [6]:
from time import clock

In [7]:
import time
random.seed(int(time.time()))

In [8]:
wavelen=3500

In [9]:
from numpy.fft import rfft, irfft

### Essential Defs

In [10]:
def model_gen():
    model = Sequential()

    model.add(BatchNormalization())

    model.add((Conv1D(64, 2, activation='relu', padding='same', input_shape=(3500,4))))
    model.add(Conv1D(64, 2, activation='relu', padding='same'))
#    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Conv1D(128, 3, activation='relu', padding='same'))
    model.add(Conv1D(128, 3, activation='relu', padding='same'))
#    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Conv1D(256, 3, activation='relu', padding='same'))
    model.add(Conv1D(256, 3, activation='relu', padding='same'))
#    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
#    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

In [11]:
def shuffle_in_unison(a, b):
    rng_state = random.get_state()
    random.shuffle(a)
    random.set_state(rng_state)
    random.shuffle(b)

In [12]:
def fft_trapezoid(rise_time, flat_top , tau):
    length = wavelen
    length = length+2*rise_time+flat_top;
    p = zeros(length)
    s = zeros(length)
    input2 = zeros(length)
    p[0] = s[0] = input2[0] = 0.;
    for i in range(1,length):
        input2[i] = s[i] = 0.;
    input2[1]=1.;
    tau = 1/(exp(1./tau)-1);
    for i in range(1,length):
        if i>=2*rise_time+flat_top:
            d = input2[i]-input2[i-rise_time]-input2[i-rise_time-flat_top]+input2[i-2*rise_time-flat_top]
        else:
            if i>=rise_time+flat_top:
                d = input2[i]-input2[i-rise_time]-input2[i-rise_time-flat_top]
            else:
                if i>=rise_time:
                    d = input2[i]-input2[i-rise_time]
                else:
                    d = input2[i];
        p[i] = p[i-1]+d;
        s[i] = s[i-1]+p[i]+tau*d;
    for i in range(length):
        s[i] = s[i]/(rise_time*tau);
    
    res = fft.rfft(s)
    return res[:-(flat_top+rise_time)]

In [13]:
def fft_cusp(shape_time, tau):
    length=wavelen
    k=2*shape_time+1;
    length = length+k;  
    m2=1.; 
    m1=m2/(exp(1./tau)-1.);
    p = empty(length)
    q = empty(length)
    s = empty(length)
    input_ = zeros(length)
    p[0]=q[0]=s[0]=input_[0]=0.
    input_[1]=1.
    dk=0.
    dl=0.
    for i in range(1,length):
        dk=0.
        dl=0.
        dk = input_[i]
        if i>=k:
            dk -= input_[i-k]

        if i>=shape_time:
            dl += input_[i-shape_time]

        if i>=shape_time+1:
            dl -= input_[i-shape_time-1]

        
        p[i] = p[i-1]+dk-k*dl;
        q[i] = q[i-1]+m2*p[i];
        s[i] = s[i-1]+q[i]+m1*p[i];
    for i in range(length):
        s[i] = s[i]/(0.5*shape_time*(shape_time+1)*m1);
    res = fft.rfft(s)
    return res[:-1*shape_time];

In [14]:
def format_data(input_X,y):    
    input_shape = (len(input_X),wavelen,4)
    X = empty(input_shape)

    short_trap=fft_trapezoid(20,0,1250)
    long_trap=fft_trapezoid(40,0,1250)
    cusp=fft_cusp(30,1250)
    
    for i in range(len(input_X)):
        X[i][:,0] = input_X[i]
        X[i][:,1] = irfft(long_trap*rfft(input_X[i]))
        X[i][:,2] = irfft(short_trap*rfft(input_X[i]))
        X[i][:,3] = irfft(cusp*rfft(input_X[i]))
    shuffle_in_unison(X,y)
    X = X.reshape(len(input_X),wavelen,4)
    return X,y

In [15]:
def gen_data():
    sys('../data_generator/bin/pulse_generation > my_data.dat')
    my_data = loadtxt('my_data.dat')
    my_labels = append(zeros(int(len(my_data)/2)),ones(int(len(my_data)/2)))
    my_data, my_labels = format_data(my_data, my_labels)
    sys('rm my_data.dat')
    return my_data, my_labels

### Comparison

In [16]:
model = model_gen()

In [17]:
X, y = gen_data()

In [18]:
y.shape

(36000,)

In [19]:
X.shape

(36000, 3500, 4)

In [20]:
keras.backend.get_session().run(tf.local_variables_initializer())

In [21]:
model.fit(X, y, batch_size=64, epochs=1, verbose=True)    

Epoch 1/1
36000/36000 [==============================] - 213s 6ms/step - loss: 0.4570 - acc: 0.7762


In [22]:
X, y = gen_data()

In [23]:
model.fit(X, y, batch_size=64, epochs=1, verbose=True)    

Epoch 1/1
36000/36000 [==============================] - 205s 6ms/step - loss: 0.3115 - acc: 0.8796


In [24]:
X, y = gen_data()

In [ ]:
model.fit(X, y, batch_size=64, epochs=10, verbose=True)    

Epoch 1/10
36000/36000 [==============================] - 207s 6ms/step - loss: 0.2416 - acc: 0.9091
Epoch 2/10
28544/36000 [======================>.......] - ETA: 43s - loss: 0.2306 - acc: 0.9152

In [ ]:
val_X, val_y = gen_data()

In [ ]:
model.evaluate(val_X, val_y, batch_size=64)